<a href="https://colab.research.google.com/github/meliksahb/Machine-Vision/blob/main/Deneme2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install requests supervision rfdetr opencv-python opencv-contrib-python matplotlib torch torchvision pillow numpy scikit-learn
!unzip Images.zip
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import requests
import io
from PIL import Image
import supervision as sv
from pathlib import Path
import time
import json
from datetime import datetime
from collections import Counter

# Import RF-DETR components
try:
    from rfdetr import RFDETRBase, RFDETRLarge
    from rfdetr.util.coco_classes import COCO_CLASSES
except ImportError:
    print("Please install RF-DETR: pip install rfdetr")
    exit(1)

Archive:  Images.zip
   creating: Yeni klasör/
  inflating: Yeni klasör/ChatGPT Image Jun 4, 2025, 06_01_00 PM.png  
  inflating: Yeni klasör/IMG-20250604-WA0046.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0047.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0048.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0049.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0050.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0051.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0052.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0053.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0054.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0055.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0056.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0057.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0058.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0059.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0060.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0061.jpg  
  inflating: Yeni klasör/IMG-20250604-WA0062.jpg  
  inflating: Yen

In [5]:
class RFDETRExperimentalFramework:
    """
    Experimental framework for RF‐DETR and a simpler fundamental CV pipeline.
    Synthetic generation and JSON reporting have been removed.
    The NMS in fundamental CV is the original single‐threshold `_simple_nms`.
    """

    def __init__(self, output_dir="./rf_detr_experiments"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)

        # Initialize models
        self.rf_detr_base = RFDETRBase()
        self.rf_detr_large = RFDETRLarge()

        # Placeholder results dictionary (not dumped to JSON here)
        self.results = {
            'rf_detr_base': {},
            'rf_detr_improved': {},
            'fundamental_cv': {},
            'rf_detr_base_challenging': {},
            'rf_detr_improved_challenging': {},
            'fundamental_cv_challenging': {},
            'rf_detr_base_camera': {},
            'rf_detr_improved_camera': {},
            'fundamental_cv_camera': {},
            'rf_detr_base_folder': {},
            'rf_detr_improved_folder': {},
            'fundamental_cv_folder': {}
        }

        print(f"[INIT] Experiment framework initialized. Output directory: {self.output_dir}")

    def load_test_images(self):
        """
        Load standard test images for initial experiments.
        """
        test_urls = [
            "https://media.roboflow.com/notebooks/examples/dog-2.jpeg",
            "https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/COCO_val2014_000000581781.jpg/640px-COCO_val2014_000000581781.jpg",
            "https://images.unsplash.com/photo-1544717297-fa95b6ee9643?ixlib=rb-4.0.3&auto=format&fit=crop&w=1000&q=80"
        ]

        images = []
        for i, url in enumerate(test_urls):
            try:
                response = requests.get(url)
                img = Image.open(io.BytesIO(response.content)).convert('RGB')
                images.append(img)
                img.save(self.output_dir / f"test_image_{i+1}.jpg")
            except Exception as e:
                print(f"[WARN] Failed to load image from {url}: {e}")

        return images

    def create_challenging_dataset(self):
        """
        Create a challenging dataset designed to make RF-DETR fail.
        """
        challenging_scenarios = [
            # Small objects in cluttered scenes
            "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?ixlib=rb-4.0.3&auto=format&fit=crop&w=1000&q=80",
            # Low contrast / lighting conditions
            "https://images.unsplash.com/photo-1518837695005-2083093ee35b?ixlib=rb-4.0.3&auto=format&fit=crop&w=1000&q=80",
            # Motion blur
            "https://images.unsplash.com/photo-1449824913935-59a10b8d2000?ixlib=rb-4.0.3&auto=format&fit=crop&w=1000&q=80",
            # Occlusion
            "https://images.unsplash.com/photo-1601758228041-f3b2795255f1?ixlib=rb-4.0.3&auto=format&fit=crop&w=1000&q=80"
        ]

        challenging_images = []
        for i, url in enumerate(challenging_scenarios):
            try:
                response = requests.get(url)
                img = Image.open(io.BytesIO(response.content)).convert('RGB')
                challenging_images.append(img)
                img.save(self.output_dir / f"challenging_image_{i+1}.jpg")
            except Exception as e:
                print(f"[WARN] Failed to load challenging image from {url}: {e}")

        return challenging_images

    def test_rf_detr_original(self, images, threshold=0.5, image_prefix=""):
        """
        Test original RF-DETR performance. Saves annotated images and records results.
        """
        results = []

        for i, image in enumerate(images):
            start_time = time.time()

            detections_base = self.rf_detr_base.predict(image, threshold=threshold)
            inference_time = time.time() - start_time

            labels = [
                f"{COCO_CLASSES[class_id]} {conf:.2f}"
                for class_id, conf in zip(detections_base.class_id, detections_base.confidence)
            ]

            annotated_image = image.copy()
            annotated_image = sv.BoxAnnotator().annotate(annotated_image, detections_base)
            annotated_image = sv.LabelAnnotator().annotate(annotated_image, detections_base, labels)
            annotated_image.save(self.output_dir / f"rf_detr_original_{image_prefix}result_{i+1}.jpg")

            result = {
                'image_id': i + 1,
                'detections': int(len(detections_base.class_id)),
                'inference_time': float(inference_time),
                'confidence_scores': [float(c) for c in detections_base.confidence.tolist()] if len(detections_base.confidence) > 0 else [],
                'classes_detected': [COCO_CLASSES[cid] for cid in detections_base.class_id] if len(detections_base.class_id) > 0 else []
            }
            results.append(result)

        if image_prefix == "challenging_":
            self.results['rf_detr_base_challenging'] = results
        elif image_prefix == "camera_":
            self.results['rf_detr_base_camera'] = results
        elif image_prefix == "folder_":
            self.results['rf_detr_base_folder'] = results
        else:
            self.results['rf_detr_base'] = results

        return results

    def implement_rf_detr_improvements(self, images, image_prefix=""):
        """
        Implement improved RF-DETR (test-time augmentation + confidence calibration + weighted voting + class NMS).
        """
        results = []

        for i, image in enumerate(images):
            start_time = time.time()

            orig_w, orig_h = image.size
            augmented_predictions = []

            # 1) Original large model
            det_orig = self.rf_detr_large.predict(image, threshold=0.4)
            augmented_predictions.append(('orig', det_orig, 1.0))

            # 2) Horizontal flip
            img_flipped = image.transpose(Image.FLIP_LEFT_RIGHT)
            det_flipped = self.rf_detr_large.predict(img_flipped, threshold=0.4)
            if len(det_flipped.xyxy) > 0:
                flipped_boxes = det_flipped.xyxy.copy()
                flipped_boxes[:, [0, 2]] = orig_w - det_flipped.xyxy[:, [2, 0]]
                det_flipped.xyxy = flipped_boxes
                augmented_predictions.append(('flip', det_flipped, 0.9))

            # 3) Two more thresholds
            for thresh in [0.3, 0.5]:
                det_t = self.rf_detr_large.predict(image, threshold=thresh)
                weight = 0.7 if thresh == 0.3 else 0.8
                augmented_predictions.append((f'thresh_{thresh}', det_t, weight))

            # Combine everything
            all_boxes = []
            all_scores = []
            all_classes = []
            all_weights = []

            for aug_type, det, weight in augmented_predictions:
                if len(det.xyxy) > 0:
                    all_boxes.append(det.xyxy)
                    calibrated = self._calibrate_confidence(det.confidence)
                    all_scores.append(calibrated * weight)
                    all_classes.append(det.class_id)
                    all_weights.append(np.full(len(det.confidence), weight))

            if all_boxes:
                all_boxes = np.vstack(all_boxes)
                all_scores = np.concatenate(all_scores)
                all_classes = np.concatenate(all_classes)
                all_weights = np.concatenate(all_weights)

                # Class-specific NMS → gather final detections
                final_boxes, final_scores, final_classes = [], [], []
                for cls in np.unique(all_classes):
                    mask = (all_classes == cls)
                    boxes_cls = all_boxes[mask]
                    scores_cls = all_scores[mask]
                    det_temp = sv.Detections(
                        xyxy=boxes_cls,
                        confidence=scores_cls,
                        class_id=np.full(len(boxes_cls), cls, dtype=int)
                    )
                    det_nms = det_temp.with_nms(threshold=0.5)
                    if len(det_nms.xyxy) > 0:
                        final_boxes.append(det_nms.xyxy)
                        final_scores.append(det_nms.confidence)
                        final_classes.append(det_nms.class_id)

                if final_boxes:
                    final_boxes = np.vstack(final_boxes)
                    final_scores = np.concatenate(final_scores)
                    final_classes = np.concatenate(final_classes)

                    mask_keep = final_scores >= 0.5
                    final_detections = sv.Detections(
                        xyxy=final_boxes[mask_keep],
                        confidence=final_scores[mask_keep],
                        class_id=final_classes[mask_keep].astype(int)
                    )
                else:
                    final_detections = sv.Detections.empty()
            else:
                final_detections = sv.Detections.empty()

            inference_time = time.time() - start_time

            labels = [
                f"{COCO_CLASSES[cid]} {conf:.2f}"
                for cid, conf in zip(final_detections.class_id, final_detections.confidence)
            ]

            annotated_image = image.copy()
            if len(final_detections) > 0:
                annotated_image = sv.BoxAnnotator(color_lookup=sv.ColorLookup.CLASS).annotate(annotated_image, final_detections)
                annotated_image = sv.LabelAnnotator().annotate(annotated_image, final_detections, labels)

            annotated_image.save(self.output_dir / f"rf_detr_improved_{image_prefix}result_{i+1}.jpg")

            result = {
                'image_id': i + 1,
                'detections': int(len(final_detections.class_id)),
                'inference_time': float(inference_time),
                'confidence_scores': [float(c) for c in final_detections.confidence.tolist()] if len(final_detections.confidence) > 0 else [],
                'classes_detected': [COCO_CLASSES[cid] for cid in final_detections.class_id] if len(final_detections.class_id) > 0 else [],
                'improvements_applied': ['tta', 'conf_calibration', 'class_nms', 'voting']
            }
            results.append(result)

        if image_prefix == "challenging_":
            self.results['rf_detr_improved_challenging'] = results
        elif image_prefix == "camera_":
            self.results['rf_detr_improved_camera'] = results
        elif image_prefix == "folder_":
            self.results['rf_detr_improved_folder'] = results
        else:
            self.results['rf_detr_improved'] = results

        return results

    def _calibrate_confidence(self, scores: np.ndarray) -> np.ndarray:
        """
        Temperature scaling on confidence scores.
        """
        temperature = 1.5
        calibrated = scores ** (1.0 / temperature)
        if len(calibrated) > 0 and calibrated.max() > 0:
            calibrated = calibrated / calibrated.max() * scores.max()
        return calibrated

    def implement_fundamental_cv_techniques(self, images, image_prefix=""):
        """
        Fundamental CV pipeline with a single-threshold NMS (_simple_nms).
        Detectors used:
         1) HOG+SVM for full-body (person)
         2) Combined Otsu + adaptive threshold contouring
         3) Canny edges
         4) Color segmentation (HSV)
         5) SIFT + DBSCAN (textured objects)
         6) Selective Search on large images only
        Finally: apply `_simple_nms`.
        """
        results = []

        # Initialize HOG person detector (more robust than Haar cascades)
        hog = cv2.HOGDescriptor()
        hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

        for i, image in enumerate(images):
            start_time = time.time()

            img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            img_gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
            h, w = img_cv.shape[:2]

            detections_found = []

            # 1) HOG+SVM person detections
            hog_rects, hog_weights = hog.detectMultiScale(
                img_gray,
                winStride=(8, 8),
                padding=(8, 8),
                scale=1.05
            )
            for (x, y, ww, hh), weight in zip(hog_rects, hog_weights):
                if weight > 0.5:
                    detections_found.append({
                        'bbox': [x, y, x + ww, y + hh],
                        'class': 'person',
                        'confidence': float(weight),
                        'method': 'HOG'
                    })

            # 2) Combined Otsu + adaptive threshold → composite mask
            blurred = cv2.GaussianBlur(img_gray, (5, 5), 0)
            _, otsu_mask = cv2.threshold(
                blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )
            adapt_mask = cv2.adaptiveThreshold(
                blurred, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY, 11, 2
            )
            composite_mask = cv2.bitwise_or(otsu_mask, adapt_mask)

            # Morphological closing/opening to clean noise
            kernel_close = np.ones((7, 7), np.uint8)
            kernel_open = np.ones((3, 3), np.uint8)
            composite_mask = cv2.morphologyEx(
                composite_mask, cv2.MORPH_CLOSE, kernel_close, iterations=2
            )
            composite_mask = cv2.morphologyEx(
                composite_mask, cv2.MORPH_OPEN, kernel_open, iterations=1
            )

            contours_oa, _ = cv2.findContours(
                composite_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )
            for cnt in contours_oa:
                area = cv2.contourArea(cnt)
                if area < 1200:
                    continue
                x, y, ww, hh = cv2.boundingRect(cnt)
                aspect = ww / float(hh)
                if not (0.4 < aspect < 2.5):
                    continue
                hull = cv2.convexHull(cnt)
                hull_area = cv2.contourArea(hull)
                solidity = float(area) / hull_area if hull_area > 0 else 0
                if solidity < 0.8:
                    continue
                detections_found.append({
                    'bbox': [x, y, x + ww, y + hh],
                    'class': 'object',
                    'confidence': float(min(0.75, solidity)),
                    'method': 'OtsuAdaptive'
                })

            # 3) Canny edges
            v_median = np.median(img_gray)
            lower_c = int(max(0, (1.0 - 0.33) * v_median))
            upper_c = int(min(255, (1.0 + 0.33) * v_median))
            edges = cv2.Canny(blurred, lower_c, upper_c)
            edges_closed = cv2.morphologyEx(
                edges, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8), iterations=1
            )
            cnts_e, _ = cv2.findContours(
                edges_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )
            for cnt in cnts_e:
                area = cv2.contourArea(cnt)
                if area < 800:
                    continue
                x, y, ww, hh = cv2.boundingRect(cnt)
                if not (ww > 30 and hh > 30 and ww < w * 0.8 and hh < h * 0.8):
                    continue
                detections_found.append({
                    'bbox': [x, y, x + ww, y + hh],
                    'class': 'edge_object',
                    'confidence': 0.55,
                    'method': 'Canny'
                })

            # 4) Color segmentation (HSV) with tuned ranges
            hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
            color_ranges = {
                'skin': ([0, 48, 80], [20, 255, 255]),
                'red_object': ([0, 100, 100], [10, 255, 255]),
                'blue_object': ([100, 150, 0], [140, 255, 255]),
                'green_object': ([40, 40, 40], [80, 255, 255]),
                'yellow_object': ([15, 150, 150], [35, 255, 255])
            }
            for name, (lower, upper) in color_ranges.items():
                low = np.array(lower, dtype=np.uint8)
                high = np.array(upper, dtype=np.uint8)
                mask_c = cv2.inRange(hsv, low, high)
                mask_c = cv2.morphologyEx(
                    mask_c, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8), iterations=1
                )
                mask_c = cv2.morphologyEx(
                    mask_c, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2
                )
                cnts_c, _ = cv2.findContours(
                    mask_c, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                )
                for cnt in cnts_c:
                    area = cv2.contourArea(cnt)
                    if area < 600:
                        continue
                    x, y, ww, hh = cv2.boundingRect(cnt)
                    if not (ww > 30 and hh > 30 and ww < w * 0.7 and hh < h * 0.7):
                        continue
                    class_label = 'person' if name == 'skin' else 'object'
                    detections_found.append({
                        'bbox': [x, y, x + ww, y + hh],
                        'class': class_label,
                        'confidence': float(min(0.7, area / 12000)),
                        'method': 'ColorSeg'
                    })

            # 5) SIFT + DBSCAN (textured objects)
            try:
                sift = cv2.SIFT_create()
                kp, des = sift.detectAndCompute(img_gray, None)
                if des is not None and len(kp) > 25:
                    pts = np.float32([pt.pt for pt in kp])
                    from sklearn.cluster import DBSCAN
                    eps_val = min(w, h) * 0.08
                    clustering = DBSCAN(eps=eps_val, min_samples=8).fit(pts)
                    for cluster_id in set(clustering.labels_):
                        if cluster_id < 0:
                            continue
                        pts_cluster = pts[clustering.labels_ == cluster_id]
                        x_min, y_min = pts_cluster.min(axis=0)
                        x_max, y_max = pts_cluster.max(axis=0)
                        ww = x_max - x_min
                        hh = y_max - y_min
                        area = ww * hh
                        if ww > 50 and hh > 50 and area > 3000:
                            cnt = np.array([[int(px), int(py)] for px, py in pts_cluster.reshape(-1, 2)])
                            hull = cv2.convexHull(cnt)
                            hull_area = cv2.contourArea(hull)
                            solidity = float(area) / hull_area if hull_area > 0 else 0
                            if solidity > 0.75:
                                detections_found.append({
                                    'bbox': [int(x_min), int(y_min), int(x_max), int(y_max)],
                                    'class': 'textured_object',
                                    'confidence': 0.65,
                                    'method': 'SIFT'
                                })
            except Exception:
                pass

            # 6) Selective Search (only if image is large)
            if max(w, h) > 512:
                try:
                    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
                    ss.setBaseImage(img_cv)
                    ss.switchToSelectiveSearchFast()
                    rects = ss.process()[:80]
                    for (x, y, ww, hh) in rects:
                        if ww < 60 or hh < 60 or ww > w * 0.7 or hh > h * 0.7:
                            continue
                        aspect = ww / float(hh)
                        if not (0.5 < aspect < 2.0):
                            continue
                        region = img_gray[y : y + hh, x : x + ww]
                        edge_density = cv2.Canny(region, 50, 150).mean()
                        if edge_density > 18:
                            detections_found.append({
                                'bbox': [x, y, x + ww, y + hh],
                                'class': 'object',
                                'confidence': float(min(0.6, edge_density / 120)),
                                'method': 'SelectiveSearch'
                            })
                except Exception:
                    print("[WARN] Selective Search unavailable (opencv-contrib-python missing).")

            # Final: run the simple NMS (single IoU threshold)
            if detections_found:
                boxes = np.array([d['bbox'] for d in detections_found])
                scores = np.array([d['confidence'] for d in detections_found])
                keep_indices = self._simple_nms(boxes, scores, iou_threshold=0.3)
                detections_found = [detections_found[idx] for idx in keep_indices]

            inference_time = time.time() - start_time

            # Visualize
            img_vis = img_cv.copy()
            method_colors = {
                'HOG': (0, 255, 0),
                'OtsuAdaptive': (255, 0, 0),
                'Canny': (255, 255, 0),
                'ColorSeg': (0, 255, 255),
                'SIFT': (128, 0, 128),
                'SelectiveSearch': (255, 0, 255),
            }
            for det in detections_found:
                x1, y1, x2, y2 = map(int, det['bbox'])
                color = method_colors.get(det['method'], (200, 200, 200))
                cv2.rectangle(img_vis, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    img_vis,
                    f"{det['class']}({det['method']}) {det['confidence']:.2f}",
                    (x1, y1 - 8),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    1,
                    lineType=cv2.LINE_AA
                )

            img_vis_rgb = cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB)
            result_pil = Image.fromarray(img_vis_rgb)
            result_pil.save(self.output_dir / f"fundamental_cv_{image_prefix}result_{i+1}.jpg")

            counts = Counter([d['method'] for d in detections_found])
            result = {
                'image_id': i + 1,
                'detections': len(detections_found),
                'inference_time': float(inference_time),
                'detections_by_method': dict(counts)
            }
            results.append(result)

        if image_prefix == "challenging_":
            self.results['fundamental_cv_challenging'] = results
        elif image_prefix == "camera_":
            self.results['fundamental_cv_camera'] = results
        elif image_prefix == "folder_":
            self.results['fundamental_cv_folder'] = results
        else:
            self.results['fundamental_cv'] = results

        return results

    def _simple_nms(self, boxes, scores, iou_threshold=0.3):
        """
        A basic IoU‐based Non‐Maximum Suppression (NMS) implementation.
        - If zero boxes: return [].
        - If one box: return [0].
        - Otherwise, perform standard IoU‐thresholding.
        """
        if boxes.size == 0:
            return []
        # If there's only one bounding box (shape (4,) or (1,4)), return [0] directly
        if boxes.ndim == 1 or boxes.shape[0] == 1:
            return [0]

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]

        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)

            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])

            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            inter = w * h

            iou = inter / (areas[i] + areas[order[1:]] - inter + 1e-6)
            inds = np.where(iou <= iou_threshold)[0]
            order = order[inds + 1]

        return keep

    def load_camera_images(self, num_frames=5, delay=0.5):
        """
        Capture a few frames from the default webcam.
        """
        camera_images = []
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("[WARN] Could not open camera.")
            return camera_images

        for i in range(num_frames):
            ret, frame = cap.read()
            if not ret:
                print(f"[WARN] Could not read frame {i+1} from camera.")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            pil_img.save(self.output_dir / f"camera_image_{i+1}.jpg")
            camera_images.append(pil_img)

            time.sleep(delay)

        cap.release()
        return camera_images

    def load_images_from_folder(self, folder_path):
        """
        Load all images from a given folder (jpg, jpeg, png, bmp, tiff).
        """
        folder = Path(folder_path)
        loaded_images = []
        if not folder.exists() or not folder.is_dir():
            print(f"[ERROR] Folder {folder_path} does not exist or is not a directory.")
            return loaded_images

        for img_path in folder.glob("*"):
            if img_path.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]:
                try:
                    img = Image.open(img_path).convert('RGB')
                    loaded_images.append(img)
                except Exception as e:
                    print(f"[WARN] Failed to load {img_path.name}: {e}")
        return loaded_images

In [6]:
# ----------------- Main Execution -----------------
if __name__ == "__main__":
    framework = RFDETRExperimentalFramework(output_dir="./rf_detr_experiments")

    # 1. Load standard test images
    test_images = framework.load_test_images()

    # 2. Create challenging dataset
    challenging_images = framework.create_challenging_dataset()

    # 3. Capture camera images (requires a connected webcam)
    camera_images = framework.load_camera_images(num_frames=5, delay=0.5)

    # 4. Load images from a user-specified folder
    folder_path = "/content/Images"
    folder_images = framework.load_images_from_folder(folder_path)

    # 5. Run RF-DETR original on all sets
    print("[RUN] RF-DETR original on standard test images...")
    framework.test_rf_detr_original(test_images, threshold=0.5, image_prefix="standard_")

    print("[RUN] RF-DETR original on challenging images...")
    framework.test_rf_detr_original(challenging_images, threshold=0.5, image_prefix="challenging_")

    if camera_images:
        print("[RUN] RF-DETR original on camera images...")
        framework.test_rf_detr_original(camera_images, threshold=0.5, image_prefix="camera_")

    if folder_images:
        print(f"[RUN] RF-DETR original on folder images from {folder_path}...")
        framework.test_rf_detr_original(folder_images, threshold=0.5, image_prefix="folder_")

    # 6. Run RF-DETR improved
    print("[RUN] RF-DETR improved on standard test images...")
    framework.implement_rf_detr_improvements(test_images, image_prefix="standard_")

    print("[RUN] RF-DETR improved on challenging images...")
    framework.implement_rf_detr_improvements(challenging_images, image_prefix="challenging_")

    if camera_images:
        print("[RUN] RF-DETR improved on camera images...")
        framework.implement_rf_detr_improvements(camera_images, image_prefix="camera_")

    if folder_images:
        print(f"[RUN] RF-DETR improved on folder images from {folder_path}...")
        framework.implement_rf_detr_improvements(folder_images, image_prefix="folder_")

    # 7. Run fundamental CV techniques on all sets
    print("[RUN] Fundamental CV on standard test images...")
    framework.implement_fundamental_cv_techniques(test_images, image_prefix="standard_")

    print("[RUN] Fundamental CV on challenging images...")
    framework.implement_fundamental_cv_techniques(challenging_images, image_prefix="challenging_")

    if camera_images:
        print("[RUN] Fundamental CV on camera images...")
        framework.implement_fundamental_cv_techniques(camera_images, image_prefix="camera_")

    if folder_images:
        print(f"[RUN] Fundamental CV on folder images from {folder_path}...")
        framework.implement_fundamental_cv_techniques(folder_images, image_prefix="folder_")

    print("[DONE] All experiments completed.")

Loading pretrain weights
Loading pretrain weights
[INIT] Experiment framework initialized. Output directory: rf_detr_experiments
[WARN] Failed to load image from https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/COCO_val2014_000000581781.jpg/640px-COCO_val2014_000000581781.jpg: cannot identify image file <_io.BytesIO object at 0x7a13a060b010>
[WARN] Could not open camera.
[RUN] RF-DETR original on standard test images...
[RUN] RF-DETR original on challenging images...
[RUN] RF-DETR original on folder images from /content/Images...
[RUN] RF-DETR improved on standard test images...
[RUN] RF-DETR improved on challenging images...
[RUN] RF-DETR improved on folder images from /content/Images...
[RUN] Fundamental CV on standard test images...
[WARN] Selective Search unavailable (opencv-contrib-python missing).
[WARN] Selective Search unavailable (opencv-contrib-python missing).
[RUN] Fundamental CV on challenging images...
[WARN] Selective Search unavailable (opencv-contrib-python mi

In [7]:
!zip -r /content/results.zip /content/rf_detr_experiments

  adding: content/rf_detr_experiments/ (stored 0%)
  adding: content/rf_detr_experiments/rf_detr_original_folder_result_53.jpg (deflated 6%)
  adding: content/rf_detr_experiments/fundamental_cv_folder_result_6.jpg (deflated 3%)
  adding: content/rf_detr_experiments/rf_detr_original_folder_result_39.jpg (deflated 2%)
  adding: content/rf_detr_experiments/fundamental_cv_folder_result_3.jpg (deflated 9%)
  adding: content/rf_detr_experiments/fundamental_cv_folder_result_50.jpg (deflated 10%)
  adding: content/rf_detr_experiments/rf_detr_improved_folder_result_53.jpg (deflated 7%)
  adding: content/rf_detr_experiments/rf_detr_improved_folder_result_49.jpg (deflated 5%)
  adding: content/rf_detr_experiments/rf_detr_improved_standard_result_1.jpg (deflated 1%)
  adding: content/rf_detr_experiments/rf_detr_improved_folder_result_48.jpg (deflated 12%)
  adding: content/rf_detr_experiments/rf_detr_original_folder_result_46.jpg (deflated 1%)
  adding: content/rf_detr_experiments/rf_detr_original